<a href="https://colab.research.google.com/github/TeamRaccoon/Model/blob/master/electra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install tokenizers

In [ ]:
import os
import random
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore')

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import sys
sys.path.append("/gdrive/My Drive/colab/electra/transformers")

from activations import get_activation
from configuration_electra import ElectraConfig
from tokenization_electra import ElectraTokenizer
from modeling_electra import ElectraPreTrainedModel, ElectraModel
from optimization import AdamW, get_linear_schedule_with_warmup


class ElectraForSequenceClassification(ElectraPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        # 분류할 라벨의 개수
        self.num_labels = config.num_labels

        # ELECTRA 모델
        self.electra = ElectraModel(config)

        self.dropout = nn.Dropout(config.hidden_dropout_prob)

        self.linear_1 = nn.Linear(config.hidden_size, config.hidden_size)
        self.linear_2 = nn.Linear(config.hidden_size, config.num_labels)

        self.softmax = nn.Softmax(dim=-1)
        
        self.biLSTM = nn.LSTM(input_size=config.hidden_size, hidden_size=int(config.hidden_size/2), num_layers=1, dropout=config.hidden_dropout_prob, batch_first=True, bidirectional=True)
        self.RNN = nn.RNN(input_size=config.hidden_size, hidden_size=int(config.hidden_size/2), nonlinearity='relu', dropout=config.hidden_dropout_prob, batch_first=True, bidirectional=True)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None):
        discriminator_hidden_states = self.electra(input_ids, attention_mask, token_type_ids)

        # (batch_size, max_length, hidden_size)
        discriminator_hidden_states = discriminator_hidden_states[0]

        lstm_output, (hidden, cell) = self.biLSTM(discriminator_hidden_states)

        # (batch_size, max_length, hidden_size) -> (batch_size, hidden_size)
        cls_output = lstm_output[:,0,:]

        cls_output = self.dropout(cls_output)

        # (batch_size, hidden_size) -> (batch_size, hidden_size)
        cls_output = self.linear_1(cls_output)
        cls_output = get_activation("gelu")(cls_output)
        cls_output = self.dropout(cls_output)

        # (batch_size, hidden_size) -> (batch_size, num_labels)
        cls_output = self.linear_2(cls_output)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(cls_output, labels)

            return loss, self.softmax(cls_output)
        else:
            return self.softmax(cls_output)

In [ ]:
import torch
import numpy as np
from tqdm import tqdm


# 학습 or 평가 데이터를 읽어 리스트에 저장
def read_data(file_path):
    with open(file_path, "r", encoding="utf8") as inFile:
        lines = inFile.readlines()

    datas = []
    for index, line in enumerate(tqdm(lines, desc="read_data")):
        # 입력 문장을 \t으로 분리
        pieces = line.strip().split("\t")

        # 데이터의 형태가 올바른지 체크
        assert len(pieces) == 3

        if(index == 0):
            continue

        id, sequence, label = pieces[0], pieces[1], int(pieces[2])
        datas.append((id, sequence, label))

    return datas


def convert_data2dataset(datas, tokenizer, max_length):
    total_input_ids, total_attention_mask, total_token_type_ids, total_labels = [], [], [], []
    for index, data in enumerate(tqdm(datas, desc="convert_data2dataset")):
        _, sequence, label = data
        tokens = tokenizer.tokenize(sequence)

        tokens = ["[CLS]"] + tokens
        tokens = tokens[:max_length-1]
        tokens.append("[SEP]")

        input_ids = [tokenizer._convert_token_to_id(token) for token in tokens]
        assert len(input_ids) <= max_length

        attention_mask = [1] * len(input_ids)
        token_type_ids = [0] * len(input_ids)

        padding = [0] * (max_length - len(input_ids))

        input_ids += padding
        attention_mask += padding
        token_type_ids += padding

        total_input_ids.append(input_ids)
        total_attention_mask.append(attention_mask)
        total_token_type_ids.append(token_type_ids)
        total_labels.append(label)

        if(index < 2):
            print("*** Example ***")
            print("sequence : {}".format(sequence))
            print("tokens: {}".format(" ".join([str(x) for x in tokens])))
            print("input_ids: {}".format(" ".join([str(x) for x in total_input_ids[-1]])))
            print("attention_mask: {}".format(" ".join([str(x) for x in total_attention_mask[-1]])))
            print("token_type_ids: {}".format(" ".join([str(x) for x in total_token_type_ids[-1]])))
            print("label: {}".format(total_labels[-1]))
            print()

    total_input_ids = torch.tensor(total_input_ids, dtype=torch.long)
    total_attention_mask = torch.tensor(total_attention_mask, dtype=torch.long)
    total_token_type_ids = torch.tensor(total_token_type_ids, dtype=torch.long)
    total_labels = torch.tensor(total_labels, dtype=torch.long)

    dataset = TensorDataset(total_input_ids, total_attention_mask, total_token_type_ids, total_labels)

    return dataset

In [ ]:
def do_train(config, electra_model, optimizer, scheduler, train_dataloader, epoch, global_step):

    # batch 단위 별 loss를 담을 리스트
    losses = []
    # 모델의 출력 결과와 실제 정답값을 담을 리스트
    total_predicts, total_corrects = [], []
    for step, batch in enumerate(tqdm(train_dataloader, desc="do_train(epoch_{})".format(epoch))):

        batch = tuple(t.cuda() for t in batch)
        input_ids, attention_mask, token_type_ids, labels = batch[0], batch[1], batch[2], batch[3]

        # 입력 데이터에 대한 출력과 loss 생성
        loss, predicts = electra_model(input_ids, attention_mask, token_type_ids, labels)

        predicts = predicts.argmax(dim=-1)
        predicts = predicts.cpu().detach().numpy().tolist()
        labels = labels.cpu().detach().numpy().tolist()

        total_predicts += predicts
        total_corrects += labels

        if config["gradient_accumulation_steps"] > 1:
            loss = loss / config["gradient_accumulation_steps"]

        # loss 값으로부터 모델 내부 각 매개변수에 대하여 gradient 계산
        loss.backward()
        losses.append(loss.data.item())

        if (step + 1) % config["gradient_accumulation_steps"] == 0 or \
                (len(train_dataloader) <= config["gradient_accumulation_steps"] and (step + 1) == len(train_dataloader)):

            torch.nn.utils.clip_grad_norm_(electra_model.parameters(), config["max_grad_norm"])

            # 모델 내부 각 매개변수 가중치 갱신
            optimizer.step()
            scheduler.step()

            # 변화도를 0으로 변경
            electra_model.zero_grad()
            global_step += 1

    # 정확도 계산
    accuracy = accuracy_score(total_corrects, total_predicts)

    return accuracy, np.mean(losses), global_step


def do_evaluate(electra_model, test_dataloader, mode):
    # 모델의 입력, 출력, 실제 정답값을 담을 리스트
    total_input_ids, total_predicts, total_corrects = [], [], []
    for step, batch in enumerate(tqdm(test_dataloader, desc="do_evaluate")):

        batch = tuple(t.cuda() for t in batch)
        input_ids, attention_mask, token_type_ids, labels = batch[0], batch[1], batch[2], batch[3]

        # 입력 데이터에 대한 출력 결과 생성
        predicts = electra_model(input_ids, attention_mask, token_type_ids)

        predicts = predicts.argmax(dim=-1)
        predicts = predicts.cpu().detach().numpy().tolist()
        labels = labels.cpu().detach().numpy().tolist()
        input_ids = input_ids.cpu().detach().numpy().tolist()

        total_predicts += predicts
        total_corrects += labels
        total_input_ids += input_ids

    # 정확도 계산
    accuracy = accuracy_score(total_corrects, total_predicts)

    if(mode == "train"):
        return accuracy
    else:
        return accuracy, total_input_ids, total_predicts, total_corrects


def train(config):
    # electra config 객체 생성
    electra_config = ElectraConfig.from_pretrained("/gdrive/My Drive/Model/model/pretrained/1st_commit", num_labels=config["num_labels"], cache_dir=config["cache_dir_path"])
    
    # electra tokenizer 객체 생성
    electra_tokenizer = ElectraTokenizer.from_pretrained("/gdrive/My Drive/Model/model/pretrained/1st_commit", do_lower_case=False, cache_dir=config["cache_dir_path"])

    # electra model 객체 생성
    electra_model = ElectraForSequenceClassification.from_pretrained("/gdrive/My Drive/Model/model/pretrained/1st_commit", config=electra_config, cache_dir=config["cache_dir_path"])

    electra_model.cuda()

    # 학습 데이터 읽기
    train_datas = read_data(file_path=config["train_data_path"])

    # 학습 데이터 전처리
    train_dataset = convert_data2dataset(datas=train_datas, tokenizer=electra_tokenizer, max_length=config["max_length"])

    # 학습 데이터를 batch 단위로 추출하기 위한 DataLoader 객체 생성
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=config["batch_size"])

    # 평가 데이터 읽기
    test_datas = read_data(file_path=config["test_data_path"])

    # 평가 데이터 전처리
    test_dataset = convert_data2dataset(datas=test_datas, tokenizer=electra_tokenizer, max_length=config["max_length"])

    # 평가 데이터를 batch 단위로 추출하기 위한 DataLoader 객체 생성
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=100)

    # 전체 학습 횟수(batch 단위)
    t_total = len(train_dataloader) // config["gradient_accumulation_steps"] * config["epoch"]

    # 모델 학습을 위한 optimizer
    optimizer = AdamW(electra_model.parameters(), lr=config["learning_rate"])
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=config["warmup_steps"], num_training_steps=t_total)

    if os.path.isfile(os.path.join(config["model_dir_path"], "optimizer.pt")) and os.path.isfile(
            os.path.join(config["model_dir_path"], "scheduler.pt")):
        # 기존에 학습했던 optimizer와 scheduler의 정보 불러옴
        optimizer.load_state_dict(torch.load(os.path.join(config["model_dir_path"], "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(config["model_dir_path"], "scheduler.pt")))

    global_step = 0
    electra_model.zero_grad()
    max_test_accuracy = 0
    for epoch in range(config["epoch"]):
        electra_model.train()

        # 학습 데이터에 대한 정확도와 평균 loss
        train_accuracy, average_loss, global_step = do_train(config=config, electra_model=electra_model,
                                                             optimizer=optimizer, scheduler= scheduler,
                                                             train_dataloader=train_dataloader,
                                                             epoch=epoch+1, global_step=global_step)

        print("train_accuracy : {}\taverage_loss : {}\n".format(round(train_accuracy, 4), round(average_loss, 4)))

        electra_model.eval()

        # 평가 데이터에 대한 정확도
        test_accuracy = do_evaluate(electra_model=electra_model, test_dataloader=test_dataloader, mode=config["mode"])

        print("test_accuracy : {}\n".format(round(test_accuracy, 4)))

        # 현재의 정확도가 기존 정확도보다 높은 경우 모델 파일 저장
        if(max_test_accuracy < test_accuracy):
            max_test_accuracy = test_accuracy

            output_dir = os.path.join(config["model_dir_path"], "checkpoint-{}".format(global_step))
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            print("save model in checkpoint-{}\n".format(global_step))
            
            electra_config.save_pretrained(output_dir)
            electra_tokenizer.save_pretrained(output_dir)
            electra_model.save_pretrained(output_dir)
            torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
            torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))

In [ ]:
def show_result(total_input_ids, total_predicts, total_corrects, tokenizer):
    for index, input_ids in enumerate(total_input_ids):
        tokens = [tokenizer._convert_id_to_token(input_id) for input_id in input_ids]

        # [CLS] 토큰 제거
        tokens = tokens[1:]

        # [SEP] 토큰 제거
        tokens = tokens[:tokens.index("[SEP]")]

        # 입력 sequence 복원
        sequence = tokenizer.convert_tokens_to_string(tokens)

        predict, correct = total_predicts[index], total_corrects[index]
        if(predict == 0):
            predict = "negative"
        else:
            predict = "positive"

        if (correct == 0):
            correct = "negative"
        else:
            correct = "positive"

        print("sequence : {}".format(sequence))
        print("predict : {}".format(predict))
        print("correct : {}".format(correct))
        print()

def test(config):
    # electra config 객체 생성
    electra_config = ElectraConfig.from_pretrained(os.path.join(config["model_dir_path"], "checkpoint-{}".format(config["checkpoint"])),
                                                   num_labels=config["num_labels"],
                                                   cache_dir=None)
    
    # electra tokenizer 객체 생성
    electra_tokenizer = ElectraTokenizer.from_pretrained(os.path.join(config["model_dir_path"], "checkpoint-{}".format(config["checkpoint"])),
                                                         do_lower_case=False,
                                                         cache_dir=None)
    
    # electra model 객체 생성
    electra_model = ElectraForSequenceClassification.from_pretrained(os.path.join(config["model_dir_path"], "checkpoint-{}".format(config["checkpoint"])),
                                                                     config=electra_config,
                                                                     cache_dir=None)
    print("\nevaluate from checkpoint-{}\n".format(config["checkpoint"]))

    electra_model.cuda()

    # 평가 데이터 읽기
    test_datas = read_data(file_path=config["test_data_path"])
    
    # 평가 데이터 전처리
    test_dataset = convert_data2dataset(datas=test_datas, tokenizer=electra_tokenizer, max_length=config["max_length"])

    # 평가 데이터를 batch 단위로 추출하기 위한 DataLoader 객체 생성
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=100)

    electra_model.eval()

    # 평가 데이터에 대한 정확도와 모델의 입력, 출력, 정답
    test_accuracy, total_input_ids, total_predicts, total_corrects = do_evaluate(electra_model=electra_model,
                                                                            test_dataloader=test_dataloader,
                                                                            mode=config["mode"])

    print("test_accuracy : {}\n".format(round(test_accuracy, 4)))

    # 10개의 평가 케이스에 대하여 모델 출력과 정답 비교
    show_result(total_input_ids=total_input_ids[:10], total_predicts=total_predicts[:10],
                total_corrects=total_corrects[:10], tokenizer=electra_tokenizer)

In [ ]:
if(__name__=="__main__"):
    root_dir = "/gdrive/My Drive/colab/electra"
    cache_dir = os.path.join(root_dir, "cache")
    output_dir = os.path.join(root_dir, "output")

    if(not os.path.exists(cache_dir)):
        os.makedirs(cache_dir)
    if (not os.path.exists(output_dir)):
        os.makedirs(output_dir)

    config = {"mode":"train",
              "train_data_path":os.path.join(root_dir, "ratings_train.txt"),
              "test_data_path": os.path.join(root_dir, "ratings_test.txt"),
              "cache_dir_path":cache_dir,
              "model_dir_path":output_dir,
              "checkpoint":2344,
              "epoch":2,
              "learning_rate":5e-5,
              "warmup_steps":0,
              "max_grad_norm":1.0,
              "batch_size":128,
              "max_length":60,
              "num_labels":2,
              "gradient_accumulation_steps":1
    }

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at /gdrive/My Drive/colab/electra/pretrained_model/wordpiece_small and are newly initialized: ['biLSTM.weight_ih_l0', 'biLSTM.weight_hh_l0', 'biLSTM.bias_ih_l0', 'biLSTM.bias_hh_l0', 'biLSTM.weight_ih_l0_reverse', 'biLSTM.weight_hh_l0_reverse', 'biLSTM.bias_ih_l0_reverse', 'biLSTM.bias_hh_l0_reverse', 'RNN.weight_ih_l0', 'RNN.weight_hh_l0', 'RNN.bias_ih_l0', 'RNN.bias_hh_l0', 'RNN.weight_ih_l0_reverse', 'RNN.weight_hh_l0_reverse', 'RNN.bias_ih_l0_reverse', 'RNN.bias_hh_l0_reverse']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
convert_data2dataset:   0%|          | 1292/602098 [00:00<01:30, 6637.22it/s]

*** Example ***
sequence : 재미는 있으나 그렇다고 엄청 재미지는 않음
tokens: [CLS] 재미 ##는 있으나 그렇다고 엄청 재미 ##지는 않 ##음 [SEP]
input_ids: 2 12792 5735 13642 16314 11988 12792 10639 3366 6137 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
attention_mask: 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
label: 1

*** Example ***
sequence : 쇼생크탈출?? 대본 작가하고 감독은 무슨 생각으로 영화만들었는지 충분히 일본이나 중국이 역사 왜곡한다고 난리치지말고 우리나라나 어린 청소년한테 잘못된 역사의식 심지말자
tokens: [CLS] 쇼 ##생 ##크 ##탈 ##출 ? ? 대 ##본 작가 ##하고 감독은 무슨 생각으로 영화 ##만 ##들 ##었는지 충분히 일본이 ##나 중국이 역사 왜곡 ##한다고 난 ##리 ##치지 ##말고 우리나라 ##나 어린 청소년 ##한테 잘못된 역사의 ##식 심 ##지 ##말 ##자 [SEP]
input_ids: 2 2989 5766 5993 5747 6024 35 35 1073 5861 13000 10546 13789 12498 27899 11001 5666 5722 21384 12764 16526 5853 14660 11514 16497 11109 761 57

convert_data2dataset:   1%|▏         | 1218/94393 [00:00<00:15, 6019.80it/s]

*** Example ***
sequence : 남편과 함께 보았는데 남편이 어렸을 적 부터 봐오던 영화라 그런지 무척 흥미 진진해 보였어요
tokens: [CLS] 남편과 함께 보 ##았는데 남편이 어렸을 적 부터 봐 ##오던 영화 ##라 그런지 무척 흥미 진 ##진 ##해 보 ##였어요 [SEP]
input_ids: 2 24637 10616 2348 15113 19213 24815 3875 15209 2374 24262 11001 5882 18695 19145 17541 4116 5679 5801 2348 31255 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
label: 1

*** Example ***
sequence : 공각기동대 문화의날에 볼만한니당
tokens: [CLS] 공 ##각 ##기 ##동 ##대 문화의 ##날 ##에 볼 ##만한 ##니당 [SEP]
input_ids: 2 317 5872 5662 5710 5789 24928 5931 5664 2355 12663 27208 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

do_evaluate:   0%|          | 2/944 [00:00<01:08, 13.66it/s]

...

test_accuracy : 0.9088

save model in checkpoint-4704



do_evaluate:   0%|          | 2/944 [00:00<01:09, 13.54it/s]

train_accuracy : 0.9535	average_loss : 0.1251



do_evaluate: 100%|██████████| 944/944 [01:10<00:00, 13.45it/s]


test_accuracy : 0.912

save model in checkpoint-9408

